<a href="https://colab.research.google.com/github/mariiacamila31/Airline_Industry/blob/main/KNN_and_Random_Forest_ML_analysis_for_the_airline_industry_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load the dataset
df = pd.read_csv('Airline_Delay_Cause.csv')
df

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",89.0,13.0,2.25,1.60,...,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",62.0,10.0,1.97,0.04,...,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,10.0,2.73,1.18,...,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",66.0,12.0,3.69,2.27,...,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",92.0,22.0,7.76,0.00,...,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12368,2023,1,YX,Republic Airline,STL,"St. Louis, MO: St Louis Lambert International",279.0,31.0,7.44,0.00,...,0.0,13.00,3.0,0.0,1775.0,258.0,0.0,704.0,0.0,813.0
12369,2023,1,YX,Republic Airline,SYR,"Syracuse, NY: Syracuse Hancock International",177.0,30.0,7.70,2.80,...,0.0,9.02,4.0,0.0,1600.0,458.0,134.0,369.0,0.0,639.0
12370,2023,1,YX,Republic Airline,TPA,"Tampa, FL: Tampa International",6.0,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12371,2023,1,YX,Republic Airline,TUL,"Tulsa, OK: Tulsa International",53.0,15.0,2.02,1.27,...,0.0,5.07,0.0,0.0,1347.0,79.0,50.0,292.0,0.0,926.0


In [ ]:
# See the info for this dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12373 entries, 0 to 12372
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 12373 non-null  int64  
 1   month                12373 non-null  int64  
 2   carrier              12373 non-null  object 
 3   carrier_name         12373 non-null  object 
 4   airport              12373 non-null  object 
 5   airport_name         12373 non-null  object 
 6   arr_flights          12355 non-null  float64
 7   arr_del15            12355 non-null  float64
 8   carrier_ct           12355 non-null  float64
 9   weather_ct           12355 non-null  float64
 10  nas_ct               12355 non-null  float64
 11  security_ct          12355 non-null  float64
 12  late_aircraft_ct     12355 non-null  float64
 13  arr_cancelled        12355 non-null  float64
 14  arr_diverted         12355 non-null  float64
 15  arr_delay            12355 non-null 

## Data cleaning

In [ ]:
# Drop the columns that will not be used for this analysis
df = df.drop(columns=['year', 'carrier_ct','carrier','late_aircraft_ct', 'weather_ct','month', 'nas_ct', 'nas_delay', 'airport_name', 'arr_cancelled','arr_diverted', 'airport', 'security_ct', 'security_delay', 'late_aircraft_delay'])
df

,carrier_name,arr_flights,arr_del15,arr_delay,carrier_delay,weather_delay
0,Endeavor Air Inc.,89.0,13.0,1375.0,71.0,761.0
1,Endeavor Air Inc.,62.0,10.0,799.0,218.0,1.0
2,Endeavor Air Inc.,62.0,10.0,766.0,56.0,188.0
3,Endeavor Air Inc.,66.0,12.0,1397.0,471.0,320.0
4,Endeavor Air Inc.,92.0,22.0,1530.0,628.0,0.0
...,...,...,...,...,...,...
12368,Republic Airline,279.0,31.0,1775.0,258.0,0.0
12369,Republic Airline,177.0,30.0,1600.0,458.0,134.0
12370,Republic Airline,6.0,0.0,0.0,0.0,0.0
12371,Republic Airline,53.0,15.0,1347.0,79.0,50.0


In [ ]:
# See the new dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12373 entries, 0 to 12372
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   carrier_name   12373 non-null  object 
 1   arr_flights    12355 non-null  float64
 2   arr_del15      12355 non-null  float64
 3   arr_delay      12355 non-null  float64
 4   carrier_delay  12355 non-null  float64
 5   weather_delay  12355 non-null  float64
dtypes: float64(5), object(1)
memory usage: 580.1+ KB


In [ ]:
# Find any missing values per
df.isnull().sum()

carrier_name      0
arr_flights      18
arr_del15        18
arr_delay        18
carrier_delay    18
weather_delay    18
dtype: int64

In [ ]:
# Handle null values in the columns by imputing a "0"
df['arr_flights'].fillna(0, inplace=True)
df['arr_del15'].fillna(0, inplace=True)
df['arr_delay'].fillna(0, inplace=True)
df['carrier_delay'].fillna(0, inplace=True)
df['weather_delay'].fillna(0, inplace=True)

#show the dataset
df

,carrier_name,arr_flights,arr_del15,arr_delay,carrier_delay,weather_delay
0,Endeavor Air Inc.,89.0,13.0,1375.0,71.0,761.0
1,Endeavor Air Inc.,62.0,10.0,799.0,218.0,1.0
2,Endeavor Air Inc.,62.0,10.0,766.0,56.0,188.0
3,Endeavor Air Inc.,66.0,12.0,1397.0,471.0,320.0
4,Endeavor Air Inc.,92.0,22.0,1530.0,628.0,0.0
...,...,...,...,...,...,...
12368,Republic Airline,279.0,31.0,1775.0,258.0,0.0
12369,Republic Airline,177.0,30.0,1600.0,458.0,134.0
12370,Republic Airline,6.0,0.0,0.0,0.0,0.0
12371,Republic Airline,53.0,15.0,1347.0,79.0,50.0


In [ ]:
# Check the changes
df.isnull().sum()

carrier_name     0
arr_flights      0
arr_del15        0
arr_delay        0
carrier_delay    0
weather_delay    0
dtype: int64

In [ ]:
# Check the data types
print(df.dtypes)

carrier_name      object
arr_flights      float64
arr_del15        float64
arr_delay        float64
carrier_delay    float64
weather_delay    float64
dtype: object


In [ ]:
# Change from float to integer with 32 bits to save memory for preprocessing
df['arr_flights'] = df['arr_flights'].astype(int)
df['arr_del15'] = df['arr_del15'].astype(int)
df['arr_delay'] = df['arr_delay'].astype(int)
df['carrier_delay'] = df['carrier_delay'].astype(int)
df['weather_delay'] = df['weather_delay'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12373 entries, 0 to 12372
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   carrier_name   12373 non-null  object
 1   arr_flights    12373 non-null  int64 
 2   arr_del15      12373 non-null  int64 
 3   arr_delay      12373 non-null  int64 
 4   carrier_delay  12373 non-null  int64 
 5   weather_delay  12373 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 580.1+ KB


In [ ]:
# Now lets see the main statistics from the cleaned data
df.describe()

,arr_flights,arr_del15,arr_delay,carrier_delay,weather_delay
count,12373.000000,12373.000000,12373.000000,12373.000000,12373.000000
mean,367.366201,82.721571,5857.123576,2086.318920,315.393276
std,1015.493432,223.478054,18111.679523,6464.278982,1070.897201
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,8.000000,461.000000,152.000000,0.000000
50%,96.000000,21.000000,1342.000000,485.000000,42.000000
75%,249.000000,58.000000,3882.000000,1553.000000,219.000000
max,20084.000000,4142.000000,438783.000000,162563.000000,25206.000000


The most important statistic from above is the **mean**:
* Notice how average delays caused by the weather (315.39) are insignificant compared to the delays due to the carrier (2,086.32). Based on this aspect only, we can conclude that the delays reflect mostly on the airlines' operations.



### Now we split/train/test the model to fit into the algorithms

For our first round of testing we will use a **test size of 20%** and a **training size of 80%**


In [ ]:
# Segregate features and labels into separate variables
X = df.drop(['carrier_name'], axis =1)
y = df['carrier_name']

In [ ]:
# Split into train and test sets
#we split the data in our model, we use a 20% test size and the remaining 80% will be trained
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# We import the minmax scaler from sklearn to normalize the data before feeding it into the algorithm
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

### Our first algorithm will be Random Forest

In [ ]:
# Import the libraries and module Random Forest
from sklearn.ensemble import RandomForestClassifier

# We define our model
rfc_model = RandomForestClassifier(random_state = 42)

# We fit the trained data that was rescaled as well as the y values to create our predictions
rfc_model.fit(rescaledX_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# We use the x_test that was rescaled to find the predictions of our model
y_pred = rfc_model.predict(rescaledX_test)

In [ ]:
# Model evaluation
print("Accuracy of Random Forest classifier: ", rfc_model.score(rescaledX_test, y_test).round(3))

Accuracy of random forest classifier:  0.37


## Our second algorithm is KNN

In [ ]:
# Import the library
from sklearn.neighbors import KNeighborsClassifier

# We assign the KNeighbors classifier to the kn_model variable
kn_model = KNeighborsClassifier()

# We fit the rescaled x and y trained values for our kn model
kn_model.fit(rescaledX_train, y_train)

KNeighborsClassifier()

In [ ]:
# We use the rescaled x test results to predict the y values using kn_model classifier
y_pred = kn_model.predict(rescaledX_test)

In [ ]:
# Model evaluation
print("Accuracy of KNN classifier: ", kn_model.score(rescaledX_test, y_test).round(3))

Accuracy of KNeighbors classifier:  0.325


Change of parameters: Reduce test size to **10%**

In [ ]:
# We split the data in our model, we use a 10% test size and the remaining 90% will be trained
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
# We import the minmax scaler from sklearn to normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [ ]:
# Import the libraries
from sklearn.ensemble import RandomForestClassifier

# We assign our model and use a random forest classifier to create the algorithm
rfc_model = RandomForestClassifier(random_state = 42) #we use 36 as our baseline to start the classification

# We fit the rescaled value of x_train into the random forest classifier and use the y-train value
rfc_model.fit(rescaledX_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
#we use the x_test that was rescaled to find the predictions of our model
y_pred = rfc_model.predict(rescaledX_test)

In [ ]:
# model evaluation
print("Accuracy of Random Forest classifier: ", rfc_model.score(rescaledX_test, y_test).round(3))
#our model using random forest classifier is 76% accurate. This means that logistic regression seems to be more effective for this dataset.

Accuracy of random forest classifier:  0.382


In [ ]:
#KNeightboursClassifier Model
#Import the library
from sklearn.neighbors import KNeighborsClassifier

#We assign the KNeighbors classifier to the kn_model variable
kn_model = KNeighborsClassifier()

#we fit the rescaled x and y trained values for our kn model
kn_model.fit(rescaledX_train, y_train)

KNeighborsClassifier()

In [ ]:
#We use the rescaled x test results to predict the y values using kn_model classifier
y_pred = kn_model.predict(rescaledX_test)

In [ ]:
# model evaluation
print("Accuracy of KNN classifier: ", kn_model.score(rescaledX_test, y_test).round(3))
#Our model is 77% accurate. This means that it is more effective than the random forest classifier, but less effective than the logistic regression model

Accuracy of KNeighbors classifier:  0.322


**Results**: By reducing the test size from 20% to 10% both algorithms increased their accuracy.
* We will continue to use Random Forest which produced the most accurate results = 38.2%

Evaluate the accuracy of a Random Forest

In [ ]:
# Import the libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Repeat the process of  selecting the variables, splitting/training/testing.
# The idea is to have the model condensed in one place

# Extract features (X) and target variable (y)
X = df.drop("carrier_name", axis=1)
y = df["carrier_name"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust hyperparameters

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{classification_report_str}")


Accuracy: 0.3804523424878837
Confusion Matrix:
[[ 15   5   2   2   2   4   1   0   1   1   4   7  13   2   1]
 [  0  38   6   1   3  13   7   0   4   5   1  15   0   3   4]
 [  2   4  35   7   1   5   2   0   7   3   1   7   2   9  10]
 [  3   4   4  48   5   5   0   1   3   2   3  37   2   3   6]
 [  3   6   5   2  12   6   1   0   1   1   8  20   2   1   4]
 [  7  13   0   3   8  54   4   0   0   7   4  11   1   1   1]
 [  2  12   7   0   1   2  18   0   1   2   0   4   1   4   9]
 [  2   4   0   1   0   0   2  11   0   0   0   1   3   1   1]
 [  2   2   4   4   2   0   3   0  20   2   0  11   1   3   6]
 [  1   4   5   1   6  15   2   0   0   5   5  14   0   2   4]
 [  0   4   2   4   7  12   1   0   0   4  13   6   1   1   4]
 [  2   8   4  18  10   8   4   1   2   5   5 102   2   0   5]
 [  7   1   5   1   1   5   0   0   0   1   1   0  63   1   5]
 [  4   2   9   3   0   4   3   0   2   0   1   2   0   7   7]
 [  4   1  17   7   0   2   7   1   4   1   2   5   5   2  30]]
Classif

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

conf_matrix = confusion_matrix(y_test, y_pred)

# Display the confusion matrix as a heatmap
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

**Hyperparameter tuning**: This step is meant to improve the model by manipulating the parameters to refine our results. The idea is to readjust the values that can create a more accurate model.

In [ ]:
# Import the libraries
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Extract features (X) and target variable (y)
X = df.drop("carrier_name", axis=1)
y = df["carrier_name"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300], #number of trees in a random forest
    'max_depth': [None, 10, 20, 30], #max depth for each tree
    'min_samples_split': [2, 5, 10], #min number of samples required
    'min_samples_leaf': [1, 2, 4] #min number of samples at a leaf node
}

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Create GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Use the best model to make predictions on the test set
y_pred = grid_search.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

# Print the best hyperparameters and evaluation metrics
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy: {accuracy}")


Best Hyperparameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy: 0.401453957996769
